In [8]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
import ktrain
from ktrain import text
os.environ['KERAS_BACKEND'] = 'theano'

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Add
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/диплом'

data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/диплом/export.csv', sep=',')
print(data_train.shape)

(2716, 7)


In [9]:
data = pd.DataFrame()
data['content'] = data_train['Содержание сообщения']
data['label'] = data_train['Оригинал сообщения']
data.dropna() 
data.head()

,content,label
0,"Действительно,кто эту провозгласил примадонной...",0
1,"Схем много,спасибо,но по всем связать пенсии н...",0
2,"И найдутся же дураки,кто это чмо купит.",0
3,"Из бисера делаю эти подснежники,а хотелось бы ...",0
4,"Да пристрелить уже этого педика ,из ума выжил ...",0


In [10]:
!pip3 install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.7 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.35.1-py3-none-any.whl size=25316191 sha256=35c267a3d6eb793aeece1185833867d9d877da9fdb339d356a71ffe43070cb60
  Stored in directory: /root/.cache/pip/wheels/3f/86/fe/b944f